In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

selected_line = ['6450-10-0']

### Size of the steps to group  


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '30min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).mean()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1320, 30):
            
            int_part, float_part = divmod((hour_day/60), 1)            
            
            if float_part != 0:
                hour_begin = str(int(int_part)) + ':' + '30'
                hour_end = str(int(int_part)) + ':' + '59'
                
                hour_test_begin = str(int(int_part + 1 )) + ':' + '00'
                hour_test_end = str(int(int_part + 1 )) + ':' + '29'
            else:
                hour_begin = str(int(int_part)) + ':' + '00'
                hour_end = str(int(int_part)) + ':' + '29'
                hour_test_begin = str(int(int_part)) + ':' + '30'
                hour_test_end = str(int(int_part)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

6450-10-0 MAPE(all) = 13.67%
6450-10-0 MAPE(Thresh>5) = 13.67%


In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']    


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '60min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).mean()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1320, 60):



            int_part, float_part = divmod((hour_day/60), 1)

            hour_begin = str(int(int_part)) + ':' + '00'
            hour_end = str(int(int_part )) + ':' + '59'

            hour_test_begin = str(int(int_part +  1)) + ':' + '00'

            hour_test_end = str(int(int_part +  1)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

8700-10-1 MAPE(all) = 9.84%
8700-10-1 MAPE(Thresh>5) = 9.41%
7545-10-1 MAPE(all) = 13.14%
7545-10-1 MAPE(Thresh>5) = 12.61%
7545-10-0 MAPE(all) = 12.11%
7545-10-0 MAPE(Thresh>5) = 11.62%
6450-10-1 MAPE(all) = 13.05%
6450-10-1 MAPE(Thresh>5) = 12.19%
6450-10-0 MAPE(all) = 12.55%
6450-10-0 MAPE(Thresh>5) = 12.55%
3301-10-1 MAPE(all) = 13.33%
3301-10-1 MAPE(Thresh>5) = 13.33%
2290-10-1 MAPE(all) = 11.59%
2290-10-1 MAPE(Thresh>5) = 11.59%
2290-10-0 MAPE(all) = 14.23%
2290-10-0 MAPE(Thresh>5) = 14.23%
477P-10-0 MAPE(all) = 17.95%
477P-10-0 MAPE(Thresh>5) = 17.95%


In [195]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']

### Size of the steps to group  


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '30min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).median()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1320, 30):
            
            int_part, float_part = divmod((hour_day/60), 1)            
            
            if float_part != 0:
                hour_begin = str(int(int_part)) + ':' + '30'
                hour_end = str(int(int_part)) + ':' + '59'
                
                hour_test_begin = str(int(int_part + 1 )) + ':' + '00'
                hour_test_end = str(int(int_part + 1 )) + ':' + '29'
            else:
                hour_begin = str(int(int_part)) + ':' + '00'
                hour_end = str(int(int_part)) + ':' + '29'
                hour_test_begin = str(int(int_part)) + ':' + '30'
                hour_test_end = str(int(int_part)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

8700-10-1 MAPE(all) = 8.24%
8700-10-1 MAPE(Thresh>5) = 8.24%
7545-10-1 MAPE(all) = 12.89%
7545-10-1 MAPE(Thresh>5) = 12.89%
7545-10-0 MAPE(all) = 11.16%
7545-10-0 MAPE(Thresh>5) = 10.35%
6450-10-1 MAPE(all) = 14.28%
6450-10-1 MAPE(Thresh>5) = 12.48%
6450-10-0 MAPE(all) = 13.47%
6450-10-0 MAPE(Thresh>5) = 13.47%
3301-10-1 MAPE(all) = 12.12%
3301-10-1 MAPE(Thresh>5) = 11.69%
2290-10-1 MAPE(all) = 11.97%
2290-10-1 MAPE(Thresh>5) = 10.79%
2290-10-0 MAPE(all) = 14.28%
2290-10-0 MAPE(Thresh>5) = 14.28%
477P-10-0 MAPE(all) = 17.93%
477P-10-0 MAPE(Thresh>5) = 17.93%


In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']    


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '60min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).median()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1320, 60):



            int_part, float_part = divmod((hour_day/60), 1)

            hour_begin = str(int(int_part)) + ':' + '00'
            hour_end = str(int(int_part )) + ':' + '59'

            hour_test_begin = str(int(int_part +  1)) + ':' + '00'

            hour_test_end = str(int(int_part +  1)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

8700-10-1 MAPE(all) = 9.45%
8700-10-1 MAPE(Thresh>5) = 9.10%
7545-10-1 MAPE(all) = 12.81%
7545-10-1 MAPE(Thresh>5) = 12.28%
7545-10-0 MAPE(all) = 11.88%
7545-10-0 MAPE(Thresh>5) = 11.45%
6450-10-1 MAPE(all) = 11.47%
6450-10-1 MAPE(Thresh>5) = 11.05%
6450-10-0 MAPE(all) = 12.14%
6450-10-0 MAPE(Thresh>5) = 12.14%
3301-10-1 MAPE(all) = 13.02%
3301-10-1 MAPE(Thresh>5) = 13.02%
2290-10-1 MAPE(all) = 10.43%
2290-10-1 MAPE(Thresh>5) = 10.43%
2290-10-0 MAPE(all) = 12.82%
2290-10-0 MAPE(Thresh>5) = 12.82%
477P-10-0 MAPE(all) = 17.68%
477P-10-0 MAPE(Thresh>5) = 17.68%


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']    


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '60min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).mean()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1320, 60):



            int_part, float_part = divmod((hour_day/60), 1)

            hour_begin = str(int(int_part)) + ':' + '00'
            hour_end = str(int(int_part )) + ':' + '59'

            hour_test_begin = str(int(int_part +  2)) + ':' + '00'

            hour_test_end = str(int(int_part +  2)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

8700-10-1 MAPE(all) = 14.68%
8700-10-1 MAPE(Thresh>5) = 14.27%
7545-10-1 MAPE(all) = 18.09%
7545-10-1 MAPE(Thresh>5) = 15.86%
7545-10-0 MAPE(all) = 15.80%
7545-10-0 MAPE(Thresh>5) = 15.29%
6450-10-1 MAPE(all) = 15.72%
6450-10-1 MAPE(Thresh>5) = 14.88%
6450-10-0 MAPE(all) = 15.99%
6450-10-0 MAPE(Thresh>5) = 15.99%
3301-10-1 MAPE(all) = 16.78%
3301-10-1 MAPE(Thresh>5) = 16.78%
2290-10-1 MAPE(all) = 15.31%
2290-10-1 MAPE(Thresh>5) = 15.31%
2290-10-0 MAPE(all) = 17.99%
2290-10-0 MAPE(Thresh>5) = 17.74%
477P-10-0 MAPE(all) = 21.40%
477P-10-0 MAPE(Thresh>5) = 21.40%


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']    


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '60min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).median()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1320, 60):



            int_part, float_part = divmod((hour_day/60), 1)

            hour_begin = str(int(int_part)) + ':' + '00'
            hour_end = str(int(int_part )) + ':' + '59'

            hour_test_begin = str(int(int_part +  2)) + ':' + '00'

            hour_test_end = str(int(int_part +  2)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

8700-10-1 MAPE(all) = 14.09%
8700-10-1 MAPE(Thresh>5) = 13.75%
7545-10-1 MAPE(all) = 17.61%
7545-10-1 MAPE(Thresh>5) = 15.74%
7545-10-0 MAPE(all) = 15.65%
7545-10-0 MAPE(Thresh>5) = 15.11%
6450-10-1 MAPE(all) = 14.41%
6450-10-1 MAPE(Thresh>5) = 14.01%
6450-10-0 MAPE(all) = 15.32%
6450-10-0 MAPE(Thresh>5) = 15.32%
3301-10-1 MAPE(all) = 16.49%
3301-10-1 MAPE(Thresh>5) = 16.49%
2290-10-1 MAPE(all) = 14.20%
2290-10-1 MAPE(Thresh>5) = 13.78%
2290-10-0 MAPE(all) = 16.56%
2290-10-0 MAPE(Thresh>5) = 16.56%
477P-10-0 MAPE(all) = 21.04%
477P-10-0 MAPE(Thresh>5) = 21.04%


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']    


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '60min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).mean()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1260, 60):



            int_part, float_part = divmod((hour_day/60), 1)

            hour_begin = str(int(int_part)) + ':' + '00'
            hour_end = str(int(int_part )) + ':' + '59'

            hour_test_begin = str(int(int_part +  3)) + ':' + '00'

            hour_test_end = str(int(int_part +  3)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

8700-10-1 MAPE(all) = 19.11%
8700-10-1 MAPE(Thresh>5) = 17.97%
7545-10-1 MAPE(all) = 21.93%
7545-10-1 MAPE(Thresh>5) = 20.30%
7545-10-0 MAPE(all) = 17.91%
7545-10-0 MAPE(Thresh>5) = 17.43%
6450-10-1 MAPE(all) = 18.98%
6450-10-1 MAPE(Thresh>5) = 18.49%
6450-10-0 MAPE(all) = 18.02%
6450-10-0 MAPE(Thresh>5) = 18.02%
3301-10-1 MAPE(all) = 19.47%
3301-10-1 MAPE(Thresh>5) = 19.47%
2290-10-1 MAPE(all) = 18.09%
2290-10-1 MAPE(Thresh>5) = 18.09%
2290-10-0 MAPE(all) = 21.43%
2290-10-0 MAPE(Thresh>5) = 21.18%
477P-10-0 MAPE(all) = 24.85%
477P-10-0 MAPE(Thresh>5) = 24.58%


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']    


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-1', format = '%Y-%m-%d')
#     df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
    df = df.loc[((df['exact_time'] > end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]
 
    
    frequency = '60min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).median()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
        
    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '23:00')]
    X_Temp.reset_index(drop=False, inplace=True)
    
    # X_Temp.reset_index(drop=False, inplace=True)

    MAPE_array = []

    for select_day in X_Temp['exact_time'].dt.strftime('%Y-%m-%d').unique():
        df_temp = X_Temp.loc[(X_Temp['exact_time'].dt.strftime('%Y-%m-%d') == select_day)]

        df_temp.index = df_temp['exact_time']
        for hour_day in range(640, 1260, 60):



            int_part, float_part = divmod((hour_day/60), 1)

            hour_begin = str(int(int_part)) + ':' + '00'
            hour_end = str(int(int_part )) + ':' + '59'

            hour_test_begin = str(int(int_part +  3)) + ':' + '00'

            hour_test_end = str(int(int_part +  3)) + ':' + '59'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_test_begin, hour_test_end)]
            
            if (not df_temp_time.empty) and (not df_temp_test.empty):
            
                MAPE_array.append(mean_absolute_percentage_error(df_temp_time.values[0][1:], df_temp_test.values[0][1:]))
        
    MAPE_array = np.array(MAPE_array)
    print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    MAPE_array = MAPE_array[MAPE_array < 50]
    print(line + ' MAPE(Thresh>5) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')

    

8700-10-1 MAPE(all) = 18.59%
8700-10-1 MAPE(Thresh>5) = 17.51%
7545-10-1 MAPE(all) = 21.37%
7545-10-1 MAPE(Thresh>5) = 20.12%
7545-10-0 MAPE(all) = 17.70%
7545-10-0 MAPE(Thresh>5) = 17.14%
6450-10-1 MAPE(all) = 17.91%
6450-10-1 MAPE(Thresh>5) = 17.43%
6450-10-0 MAPE(all) = 17.61%
6450-10-0 MAPE(Thresh>5) = 17.61%
3301-10-1 MAPE(all) = 19.27%
3301-10-1 MAPE(Thresh>5) = 19.27%
2290-10-1 MAPE(all) = 16.99%
2290-10-1 MAPE(Thresh>5) = 16.68%
2290-10-0 MAPE(all) = 19.76%
2290-10-0 MAPE(Thresh>5) = 19.51%
477P-10-0 MAPE(all) = 24.55%
477P-10-0 MAPE(Thresh>5) = 24.28%
